# Excel in tuning models using Amazon LinearLearner Algorithm

# Assumptions and Disclaimers
This blogpost assumes that you have already completed the following tutorials from Amazon SageMaker docuemntation:
- [Setting up](https://docs.aws.amazon.com/sagemaker/latest/dg/gs-set-up.html)
- [Create am Amazon SageMaker Notebook Instance](https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html)
- I have included 'sagemaker' in the name of my S3 bucket, "cyrusmv-sagemaker-demos' and have chosen to let any SageMaker notebook instance to access any S3 bucket with the term 'sagemaker' included in the name. This is however is not a recommended security option for production and is only useful for simplifying the flow of the blog.
- It is assumed that the reader is familiar with linear regression. If not please read part 1 of this post, Linear Regression and Binary Classification, a Friendly Introduction.



# Hyperparameter Tuning
In the previous part we used default hyperparameters:
u'epochs': u'10', u'init_bias': u'0.0', u'lr_scheduler_factor': u'0.99', **u'num_calibration_samples': u'10000000'**, u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'bias_lr_mult': u'10', u'lr_scheduler_step': u'100', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'0.00001', **u'target_recall': u'0.8'**, **u'num_models': u'32'**, u'momentum': u'0.0', u'unbias_label': u'auto', u'wd': u'0.0', u'optimizer': u'adam', u'learning_rate': u'auto', u'_kvstore': u'auto', **u'normalize_data': u'true'**, **u'binary_classifier_model_selection_criteria': u'accuracy'**, u'use_lr_scheduler': u'true', **u'target_precision': u'0.8'**, u'force_dense': u'true', u'unbias_data': u'auto', u'init_scale': u'0.07', u'bias_wd_mult': u'0', u'mini_batch_size': u'1000', u'beta_1': u'0.9', u'loss': u'auto', u'beta_2': u'0.999', u'normalize_label': u'auto', u'_num_gpus': u'auto', u'_data_format': u'record', u'positive_example_weight_mult': u'1.0', u'l1': u'0.0'}

Let us highlight a few of these parameters:
- **target_recall and target_accuracy** are both set to 80%
- **normalize_data** is already true.
- **binary_classifier_model_selection_criteria** is accuracy. I will change it to **recall_at_target_precision**. This forces the model to optimizefor highest recall at a given precision target.
- **num_models** is 32 so we know that HPO is running 32 models in parallel.
- **num_calibration_samples** is Number of observations to use from the validation dataset for model calibration (finding the best threshold).It is set to 1000000. 

for more information on linear learner hyperparmetyers pleaese refer to [Amazon SageMaker Documentaiton](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html)


In [272]:
#imports
import boto3 #AWS python SDK for accessing AWS services
import numpy as np #Array libraru with probability and statistics capabilities
import io
import sagemaker.amazon.common as smac # Amazon Sagemaker common library that includes data formats
import sagemaker #sagemaker python sdk
import os
from sagemaker.predictor import csv_serializer, json_deserializer #sagemaker prediction sdk
from sagemaker import get_execution_role


In [273]:
bucket = 'cyrusmv-sagemaker-demos'     #replace this with your own bucket
local = 'data/processed'               #replace with your local destination
dist = 'visa-kaggle/data/'             #replace with your top directory in S3 for all the data files
files = {}

role = get_execution_role() #this is SageMaker role that would be later used for authorizing SageMaker to access S3
print(role) 

sagemaker_session = sagemaker.Session()

arn:aws:iam::475933981307:role/service-role/AmazonSageMaker-ExecutionRole-20180102T172706


# Downloading Data Files from S3
We iterate over S3 subfolders recursively and when reaching a leaf, we download the file. We also append the location of the file and key to files array, so the code can be generalized based on your folder structure in S3. 

*Disclaimer: The code here is based on [this stackoverflow reference](https://stackoverflow.com/questions/31918960/boto3-to-download-all-files-from-a-s3-bucket) plus exception handling and creating a dictionary of files.*

In [274]:
def download_dir(client, resource, dist, local, bucket):
    paginator = client.get_paginator('list_objects')
    for result in paginator.paginate(Bucket=bucket, Delimiter='/', Prefix=dist):
        if result.get('CommonPrefixes') is not None:
            for subdir in result.get('CommonPrefixes'):
                download_dir(client, resource, subdir.get('Prefix'), local, bucket)
        if result.get('Contents') is not None:
            for file in result.get('Contents'):
                if not os.path.exists(os.path.dirname(local + os.sep + file.get('Key'))):
                    os.makedirs(os.path.dirname(local + os.sep + file.get('Key')))
                print('bucket: {} source file: {}; ==> local: {} \n'.format(bucket, file.get('Key'), local + os.sep + file.get('Key')))
                try:
                    dest = local + os.sep + file.get('Key')
                    key = dest.rsplit('/',1)[-1]
                    key = key.rsplit('.', 1)[0]
                    resource.meta.client.download_file(bucket, file.get('Key'),dest)
                    files[key] = dest
                except (IsADirectoryError, NotADirectoryError):
                    print('WARNING: {}/{} is a directory, skipping download operation'.format(bucket, file.get('Key')))

                    
def _start():
    client = boto3.client('s3')
    resource = boto3.resource('s3')
    download_dir(client, resource, local=local, bucket=bucket, dist=dist)
    print('\ndownload completed.')
    
_start()

files


bucket: cyrusmv-sagemaker-demos source file: visa-kaggle/data/output/linear-learner-2018-02-13-13-57-11-769/output/model.tar.gz; ==> local: data/processed/visa-kaggle/data/output/linear-learner-2018-02-13-13-57-11-769/output/model.tar.gz 

bucket: cyrusmv-sagemaker-demos source file: visa-kaggle/data/output/linear-learner-2018-02-13-13-57-48-247/output/model.tar.gz; ==> local: data/processed/visa-kaggle/data/output/linear-learner-2018-02-13-13-57-48-247/output/model.tar.gz 

bucket: cyrusmv-sagemaker-demos source file: visa-kaggle/data/test/val_data.npy; ==> local: data/processed/visa-kaggle/data/test/val_data.npy 

bucket: cyrusmv-sagemaker-demos source file: visa-kaggle/data/test/val_label.npy; ==> local: data/processed/visa-kaggle/data/test/val_label.npy 

bucket: cyrusmv-sagemaker-demos source file: visa-kaggle/data/train/train_data.npy; ==> local: data/processed/visa-kaggle/data/train/train_data.npy 

bucket: cyrusmv-sagemaker-demos source file: visa-kaggle/data/train/train_label.

{'model.tar': 'data/processed/visa-kaggle/data/output/linear-learner-2018-02-13-13-57-48-247/output/model.tar.gz',
 'recordio-pb-data': 'data/processed/visa-kaggle/data/recordio-pb-data',
 'train_data': 'data/processed/visa-kaggle/data/train/train_data.npy',
 'train_label': 'data/processed/visa-kaggle/data/train/train_label.npy',
 'val_data': 'data/processed/visa-kaggle/data/test/val_data.npy',
 'val_label': 'data/processed/visa-kaggle/data/test/val_label.npy'}

# Loading Data into Vectors
We will need to have the train and validation data to be loaded into numpy vectors before oriessing them.

In [275]:
train_data = np.load(files['train_data'])
train_label = np.load(files['train_label'])

val_data = np.load(files['val_data'])
val_label = np.load(files['val_label'])

print("training data shape= {}; training label shape = {} \nValidation data shape= {}; validation label shape = {}".format(train_data.shape, 
                                                                        train_label.shape,
                                                                        val_data.shape,
                                                                        val_label.shape))
train_set = (train_data, train_label)
test_set = (val_data, val_label)


training data shape= (199364, 30); training label shape = (199364,) 
Validation data shape= (85443, 30); validation label shape = (85443,)


# Converting the Data
Amazon Algorithms support csv and recordio/protobuf. recordio is faster than CSV and specially in algorithms that deal with sparse matrices.
In the below snippet I am using sagemaker.amazon.core library in order to convert my numpy arrays into protobuf recordIO.

In [276]:
vectors = np.array([t.tolist() for t in train_set[0]]).astype('float32')
labels = np.array([t.tolist() for t in train_set[1]]).astype('float32')

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, vectors, labels)
buf.seek(0)

0

# Upload Training Data to S3
Now that we've created our recordIO-wrapped protobuf, we'll need to upload it to S3, so that Amazon SageMaker training can use it.

In [277]:
key = 'recordio-pb-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(dist, key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}{}'.format(bucket, dist, key)
print('uploaded training data location: {}'.format(s3_train_data))

uploaded training data location: s3://cyrusmv-sagemaker-demos/visa-kaggle/data/recordio-pb-data


Let's also setup an output S3 location for the model artifact to be uploaded to after training in complete.

In [278]:
output_location = 's3://{}/{}output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

training artifacts will be uploaded to: s3://cyrusmv-sagemaker-demos/visa-kaggle/data/output


# Training the Model with New Hyper Parameters

In [255]:
containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/linear-learner:latest',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:latest',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/linear-learner:latest',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/linear-learner:latest'}

In [284]:
sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                       role, #S3 role, so the notebook can read the data and upload the model
                                       train_instance_count=1, #number of instances for training
                                       train_instance_type='ml.c4.8xlarge', # type of training instance
                                       output_path=output_location, #s3 location for uploading trained mdoel
                                       sagemaker_session=sess)

linear.set_hyperparameters(feature_dim=30, #dataset has 30 columns (features)
                           predictor_type='binary_classifier', # we predict a binary value. it could have been regressor
                           mini_batch_size=200,
                           #making recall the selection criteria and changin calibration samples that are used for threshold setting
                           binary_classifier_model_selection_criteria='recall_at_target_precision')

linear.fit({'train': s3_train_data})

INFO:sagemaker:Creating training-job with name: linear-learner-2018-02-13-16-42-33-379


...................................................................
Docker entrypoint called with argument(s): train
[02/13/2018 16:48:03 INFO 140683661657920] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'epochs': u'10', u'init_bias': u'0.0', u'lr_scheduler_factor': u'0.99', u'num_calibration_samples': u'10000000', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'bias_lr_mult': u'10', u'lr_scheduler_step': u'100', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'0.00001', u'target_recall': u'0.8', u'num_models': u'32', u'momentum': u'0.0', u'unbias_label': u'auto', u'wd': u'0.0', u'optimizer': u'adam', u'learning_rate': u'auto', u'_kvstore': u'auto', u'normalize_data': u'true', u'binary_classifier_model_selection_criteria': u'accuracy', u'use_lr_scheduler': u'true', u'target_precision': u'0.8', u'force_dense': u'true', u'unbias_d

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.005667652155470896, "sum": 0.005667652155470896, "min": 0.005667652155470896}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540523.807105, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1518540523.807015}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.004825513667789808, "sum": 0.004825513667789808, "min": 0.004825513667789808}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540523.807182, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1518540523.807169}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.004935127349054239, "sum": 0.004935127349054239, "min": 0.004935127349054239}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540543.850331, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1518540543.850242}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.00427203314891272, "sum": 0.00427203314891272, "min": 0.00427203314891272}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540543.850408, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1518540543.850393}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, 

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.00470667933693612, "sum": 0.00470667933693612, "min": 0.00470667933693612}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540563.582167, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1518540563.582078}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.004219565484186552, "sum": 0.004219565484186552, "min": 0.004219565484186552}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540563.582244, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1518540563.58223}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "

...

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.004545867700937642, "sum": 0.004545867700937642, "min": 0.004545867700937642}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540623.147596, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 7}, "StartTime": 1518540623.147506}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.004049344359650221, "sum": 0.004049344359650221, "min": 0.004049344359650221}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540623.147672, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 7}, "StartTime": 1518540623.147659}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.004526676941722871, "sum": 0.004526676941722871, "min": 0.004526676941722871}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540643.987637, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 8}, "StartTime": 1518540643.987549}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.004051024555509169, "sum": 0.004051024555509169, "min": 0.004051024555509169}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540643.987707, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 8}, "StartTime": 1518540643.987693}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.004505495541470956, "sum": 0.004505495541470956, "min": 0.004505495541470956}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540665.74897, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 9}, "StartTime": 1518540665.748883}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.004010293081058675, "sum": 0.004010293081058675, "min": 0.004010293081058675}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540665.749038, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 9}, "StartTime": 1518540665.749025}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.004484348957871936, "sum": 0.004484348957871936, "min": 0.004484348957871936}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540687.307853, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 10}, "StartTime": 1518540687.307762}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count": 1, "max": 0.004022592038222884, "sum": 0.004022592038222884, "min": 0.004022592038222884}, "validation_binary_classification_cross_entropy": {"count": 1, "max": -Infinity, "sum": NaN, "min": Infinity}}, "EndTime": 1518540687.307922, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 10}, "StartTime": 1518540687.307909}

#metrics {"Metrics": {"training_binary_classification_cross_entropy": {"count"

===== Job Complete =====


# Hosting

In [285]:
linear_predictor = linear.deploy(initial_instance_count=1, #Initial number of instances. 
                                                           #Autoscaling can increase the number of instances.
                                 instance_type='ml.m4.xlarge') # instance type

INFO:sagemaker:Creating model with name: linear-learner-2018-02-13-17-03-24-734
INFO:sagemaker:Creating endpoint with name linear-learner-2018-02-13-16-42-33-379


-----------------------------------------------------------------------------------------------------------------------------------------!

In [286]:
type(linear_predictor)

sagemaker.predictor.RealTimePredictor

# Prediction

In [287]:
linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

In [288]:
predictions = []
for array in np.array_split(test_set[0], 100):
    result = linear_predictor.predict(array)
    predictions += [r['predicted_label'] for r in result['predictions']]

predictions = np.array(predictions)

In [289]:
import pandas as pd

pd.crosstab(test_set[1], predictions, rownames=['actuals'], colnames=['predictions'])

predictions,0.0,1.0
actuals,,
0.0,85268,28
1.0,28,119


In [291]:
print("false positive after Hyper-Parameter change = {}".format(119/(28+119)))
print("false positive before Hyper-Parameter change = {}".format(118/(29+118)))


false positive after Hyper-Parameter change = 0.8095238095238095
false positive before Hyper-Parameter change = 0.8027210884353742


# Analyzing the Results
recall on fraud is this mode is about 81% as opposed to 80% with default parameters. The improvement is less that 1% while we lost about 10% in terms of accuracy as it is also evident from the confusion matrix below.
An important fact to notice is that from parallel models, in this model, model #8 and in the model with default values model #12, yielded the results. This is testament to the power of HPO, which SageMaker provides out of the box. Running 32 models in parallel has produced better result compared to my attempts to improve the results based on manual tuning.

| After Changing Hyper-Parameters | Before Changing Hyper-Parameters|
|:--------------------------------|:--------------------------------|
| model: 8                        | model: 12                       |
| threshold: 0.019479             | threshold: 0.028                |
| score: 0.808696                 | score: 0.999418                 |


# Concolusions
There are strong benefits in using Amazon SageMaker and out-of-the-box algorithms. 

We have observed how easily we can build an end-to-end data science pipeline without worrying about operational issues. 

pre-trained algorithms provide you with benefits of transfer learning and the ability to start from a trained model and improve the model for your data. This will reduce training time and thus reducing cost of training.

HPO (Hyper-Parameter Optimizer) permits running several models in parallel and picking the best one automatically. This significantly redues experiment time by taking over tedious tasks of tweaking parameters and comparing results.


# (optional) Delete the endpoint
f you're ready to be done with this notebook, please run the delete_endpoint line in the cell below. This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
linear.delete_endpoint()